# PLM NSMC Finetuning

## 0. 미션
참조
- model: https://huggingface.co/klue/roberta-base
- data
  - https://github.com/e9t/nsmc
  - https://huggingface.co/datasets/e9t/nsmc

미션
- klue/roberta-base 모델을 활용하여 문장의 감정을 긍정/부정으로 예측하는 감정분류 데이터셋인 NSMC를 fine-tuning을 해 보면서 HuggingFace 사용법 및 PLM 학습 방법을 습득합니다.
- GPU는 6.fine-tuning 과정에서만 사용 하셔도 됩니다.

## 1. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *2. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 2.1. 라이브러리 Import

In [3]:
# install datasets 라이브러리
!pip install -qq datasets

In [5]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TrainingArguments,
                          Trainer)

### 2.2. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.

In [6]:
WORKSPACE = '/content/drive/MyDrive/언어지능/nlp-practice'

In [7]:
!mkdir {WORKSPACE}/data

mkdir: cannot create directory ‘/content/drive/MyDrive/언어지능/nlp-practice/data’: File exists


In [8]:
!ls {WORKSPACE}

01.plm-nsmc-student.ipynb  02.llm-nsmc-student.ipynb  03.rag-student.ipynb  data


## 3. Dataset understanding
- Dataset의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- https://huggingface.co/docs/datasets/quickstart

In [10]:
# Hugging Face Hub에서 dataset 다운로드
# dataset = load_dataset("e9t/nsmc", trust_remote_code=True)
dataset = load_dataset("e9t/nsmc")

The repository for e9t/nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/e9t/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] ㅛ
The repository for e9t/nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/e9t/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
# dataset 구조 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [12]:
# train features 확인
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [13]:
# train num_rows 확인
dataset['train'].num_rows

150000

In [14]:
# train 10개 데이터만 확인
for i  in range(10):
    print(dataset['train'][i])

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
{'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
{'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0}
{'id': '6483659', 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'label': 1}
{'id': '5403919', 'document': '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', 'label': 0}
{'id': '7797314', 'document': '원작의 긴장감을 제대로 살려내지못했다.', 'label': 0}
{'id': '9443947', 'document': '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', 'label': 0}
{'id': '7156791', 'document': '액션이 없는데도 재미 있는 몇안되는 영화', 'label': 1}
{'id': '5912145', 'document': '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', 'label': 1}


### Mission
- Hugging Face Hub 'supark/ko-stsb' dataset을 다운로드 하세요.
- train 데이터의 features, num_rows를 확인하세요.
- train 데이터 값 10개만 출력하세요.

In [35]:
# Hugging Face Hub에서 dataset 다운로드
dataset = load_dataset("supark/ko-stsb")

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/524k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/162k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/123k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5709 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1498 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1378 [00:00<?, ? examples/s]

In [36]:
# dataset 구조 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 5709
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 1498
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 1378
    })
})

In [37]:
# train features 확인
dataset['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'score': Value(dtype='float64', id=None)}

In [38]:
# train num_rows 확인
dataset['train'].num_rows

5709

In [41]:
# train 10개 데이터만 확인
for i in range(10):
  print(dataset['train'][i])

{'sentence1': '비행기가 이륙하고 있다.', 'sentence2': '비행기가 이륙하고 있다.', 'score': 1.0}
{'sentence1': '한 남자가 큰 플루트를 연주하고 있다.', 'sentence2': '남자가 플루트를 연주하고 있다.', 'score': 0.76}
{'sentence1': '한 남자가 피자에 치즈를 뿌려놓고 있다.', 'sentence2': '한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.', 'score': 0.76}
{'sentence1': '세 남자가 체스를 하고 있다.', 'sentence2': '두 남자가 체스를 하고 있다.', 'score': 0.52}
{'sentence1': '한 남자가 첼로를 연주하고 있다.', 'sentence2': '자리에 앉은 남자가 첼로를 연주하고 있다.', 'score': 0.85}
{'sentence1': '몇몇 남자들이 싸우고 있다.', 'sentence2': '두 남자가 싸우고 있다.', 'score': 0.85}
{'sentence1': '남자가 담배를 피우고 있다.', 'sentence2': '남자가 스케이트를 타고 있다.', 'score': 0.1}
{'sentence1': '남자가 피아노를 치고 있다.', 'sentence2': '남자가 기타를 연주하고 있다.', 'score': 0.32}
{'sentence1': '한 남자가 기타를 치고 노래를 부르고 있다.', 'sentence2': '한 여성이 어쿠스틱 기타를 연주하고 노래를 부르고 있다.', 'score': 0.44000000000000006}
{'sentence1': '사람이 고양이를 천장에 던지고 있다.', 'sentence2': '사람이 고양이를 천장에 던진다.', 'score': 1.0}


## 4. Tokenizer understanding
- Tokenizer의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- https://huggingface.co/docs/tokenizers/quicktour

In [15]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# tokenizer 확인
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
# 동작 확인을 위한 문장
sentence1 = "지미 카터: 제임스 얼 지미 카터 주니어(1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다."
sentence2 = "수학: 수학(math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다."

In [18]:
# tokenizer tokens 확인
tokens = tokenizer.tokenize(sentence1)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [19]:
# tokens to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18]


In [20]:
# ids to tokens
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [21]:
# [CLS], tokens ids, [SEP] 형식으로 변환
token_ids = tokenizer.encode(sentence1)
print(token_ids)

[0, 19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18, 2]


In [22]:
# 실제 모델에 입력형식으로 변환
# 배열에 미니배치 형식으로 여러 문장을 넣을 수 있음
inputs = tokenizer([sentence1, sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    0, 19537, 20604,    30, 10073,  1411, 19537, 20604, 14577,    12,
         23489,  2440,  3633,  2429,    21,  2210,    97,    13,   793,  4186,
          4327,  3666,  2079,  1545, 24834,  2104,  3698,    12, 13528,    17,
          7561,    13, 30651,    18,     2],
        [    0,  5193,    30,  5193,    12,    80, 16012,    13,  1497,  1295,
            16,  1402,    16,  3962,    16,  4101,    16,  3908,   886,  2079,
          4453,  2069,  5778,  2259,  6204, 28674,    18,     2,     1,     1,
             1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1

### Mission
- Hugging Face Hub에서 'klue/bert-base' tokenizer를 다운로드 하세요.
- 위 문장을 tokeinze 해 보세요.

In [24]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [25]:
# tokenizer 확인
tokenizer

BertTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [26]:
# tokenizer tokens 확인
tokens = tokenizer.tokenize(sentence1)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [27]:
# tokens to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18]


In [28]:
# ids to tokens
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [29]:
# tokens ids 형식으로 변환
token_ids = tokenizer.encode(sentence1)
print(token_ids)

[2, 19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18, 3]


In [42]:
# 실제 모델에 입력형식으로 변환
# 배열에 미니배치 형식으로 여러 문장을 넣을 수 있음
inputs = tokenizer([sentence1, sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    2, 19537, 20604,    30, 10073,  1411, 19537, 20604, 14577,    12,
         23489,  2440,  3633,  2429,    21,  2210,    97,    13,   793,  4186,
          4327,  3666,  2079,  1545, 24834,  2104,  3698,    12, 13528,    17,
          7561,    13, 30651,    18,     3],
        [    2,  5193,    30,  5193,    12,    80, 16012,    13,  1497,  1295,
            16,  1402,    16,  3962,    16,  4101,    16,  3908,   886,  2079,
          4453,  2069,  5778,  2259,  6204, 28674,    18,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1

## 5. Model understanding
- Model의 동작 및 사용 방법을 이해하기 위한 과정입니다.

In [43]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
# Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",
                                                           num_labels=2)  # 예측할 class 개수

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
# model 확인
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [46]:
sentence1 = '아 더빙.. 진짜 짜증나네요 목소리'
sentence2 = '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [47]:
# 입력 생성
inputs = tokenizer([sentence1,
                    sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    0,  1376,   831,  2604,    18,    18,  4229,  9801,  2075,  2203,
          2182,  4243,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  1963,    18,    18,    18, 11811,  2178,  2088, 28883, 16516,
          2776,    18,    18,    18,    18, 10737,  2156,  2015,  2446,  2232,
          6758,  2118,  1380,  6074,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [48]:
# 추론
# 미세조정(finetuning)을 하지 않았기 때문에 현재는 답변을 신뢰 할 수 없음)
results = model(**inputs)
results

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2692, -0.0704],
        [ 0.2703, -0.0743]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [49]:
# logits 확인
results.logits

tensor([[ 0.2692, -0.0704],
        [ 0.2703, -0.0743]], grad_fn=<AddmmBackward0>)

### Mission
- Hugging Face Hub에서 'klue/bert-base' model 다운로드 하세요.
- 위 문장을 추론해 보세요.

In [52]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [53]:
# Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",
                                                           num_labels=2)  # 예측할 class 개수

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# model 확인
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [56]:
sentence1 = '아 더빙.. 진짜 짜증나네요 목소리'
sentence2 = '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [57]:
# 입력 생성
inputs = tokenizer([sentence1,
                    sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    2,  1376,   831,  2604,    18,    18,  4229,  9801,  2075,  2203,
          2182,  4243,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [    2,  1963,    18,    18,    18, 11811,  2178,  2088, 28883, 16516,
          2776,    18,    18,    18,    18, 10737,  2156,  2015,  2446,  2232,
          6758,  2118,  1380,  6074,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [60]:
# 추론
# 미세조정(finetuning)을 하지 않았기 때문에 현재는 답변을 신뢰 할 수 없음)
result = model(**inputs)

In [61]:
# logits 확인
results.logits

tensor([[ 0.2692, -0.0704],
        [ 0.2703, -0.0743]], grad_fn=<AddmmBackward0>)

## 6. Finetuning
- PLM finetuning 방법을 이해하기 위한 과정입니다.
- TODO 코드를 완성하고 학습을 완료 하세요.

In [63]:
# TODO: Hugging Face Hub에서 dataset 다운로드
dataset = load_dataset("e9t/nsmc")

In [64]:
# TODO: Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [65]:
# TODO: Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",
                                                           num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
# dataset 선언
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]['document'], self.dataset[idx]['label']

train_dataset = NSMCDataset(dataset['train'])
test_dataset = NSMCDataset(dataset['test'])

In [67]:
# collator 선언
class NSMCCollator:
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch):
        texts, labels = zip(*batch)

        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return_value = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": torch.tensor(labels, dtype=torch.long),
        }

        return return_value

nsmc_collator = NSMCCollator(tokenizer)

In [68]:
# epoch 당 step 수 계산
steps_per_epoch = int(np.ceil(len(dataset['train']) / 128))
steps_per_epoch

1172

In [69]:
# Train arguments 선언
training_args = TrainingArguments(
    output_dir=f"{WORKSPACE}/checkpoint/klue-roberta-base",
    overwrite_output_dir=True,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    warmup_steps=1000,
    num_train_epochs=3,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    # bf16=config.fp16,
    # bf16_full_eval=config.fp16,
    fp16=True,
    fp16_full_eval=True,
    half_precision_backend=True,
    load_best_model_at_end=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [70]:
# trainer 선언
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=nsmc_collator,
    )

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [71]:
# train
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.315600,0.249306
2,0.212100,0.224569
3,0.131300,0.249899


TrainOutput(global_step=3516, training_loss=0.21962596559144804, metrics={'train_runtime': 2139.1589, 'train_samples_per_second': 210.363, 'train_steps_per_second': 1.644, 'total_flos': 1.997944487762784e+16, 'train_loss': 0.21962596559144804, 'epoch': 3.0})

## 7. Evaluation
- PLM finetuning 후 평가 방법을 이해하기 위한 과정입니다.
- TODO 코드를 완성하고 학습을 완료 하세요.

In [74]:
# TODO: Hugging Face Hub에서 dataset 다운로드
dataset = load_dataset("e9t/nsmc")

In [75]:
# 저장된 결과 확인
!ls {WORKSPACE}/checkpoint/klue-roberta-base

checkpoint-1172  checkpoint-2344  checkpoint-3516


In [76]:
# TODO: 가장 성능이 좋은 경로를 입력하세요.
best_fn = f"{WORKSPACE}/checkpoint/klue-roberta-base/checkpoint-2344"

In [77]:
# TODO: 저장된 tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(best_fn)

In [78]:
# TODO: 저장된 model 로드
model = AutoModelForSequenceClassification.from_pretrained(best_fn,
                                                           num_labels=2)

In [79]:
# dataset 선언
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]['document'], self.dataset[idx]['label']

train_dataset = NSMCDataset(dataset['train'])
test_dataset = NSMCDataset(dataset['test'])

In [80]:
# collator 선언
class NSMCCollator:
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch):
        texts, labels = zip(*batch)

        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return_value = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": torch.tensor(labels, dtype=torch.long),
        }

        return return_value

nsmc_collator = NSMCCollator(tokenizer)

In [81]:
# test loader 생성
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=nsmc_collator
)

In [82]:
# gpu 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [83]:
total_correct_cnt = 0
total_sample_cnt = 0

model.to(device)
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        output = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
        )
        probs = torch.functional.F.softmax(output.logits, dim=-1)
        prob, pred = torch.max(probs, dim=-1)

        correct_cnt = (pred.cpu() == batch["labels"]).sum().item()
        sample_cnt = len(batch["labels"])

        total_correct_cnt += correct_cnt
        total_sample_cnt += sample_cnt

print(f"Test Accuracy: {total_correct_cnt / total_sample_cnt * 100:.2f}%")
print(f"Correct / Total: {total_correct_cnt} / {total_sample_cnt}")

  0%|          | 0/391 [00:00<?, ?it/s]

Test Accuracy: 90.80%
Correct / Total: 45399 / 50000


In [84]:
# idx -> label
idx2label = {0: '부정', 1: '긍정'}

In [85]:
# 랜덤하게 10개 셈플 결과 확인
idxs = np.random.randint(0, dataset['test'].num_rows, 10)

with torch.no_grad():
    for idx in idxs:
        document = dataset['test'][int(idx)]['document']
        # inputs 생성
        inputs = tokenizer(
                    document,
                    truncation=True,
                    max_length=256,
                    return_tensors="pt",
                ).to(device)
        # 추론
        logit = model(**inputs).logits[0]
        prob = F.softmax(logit, dim=-1)
        # 가장 높은 확률을 정답으로 예측
        y = prob.argmax(dim=-1)
        # 출력
        print(f"{idx2label[y.item()]}\t{prob[y].item():.4f}\t{document}")

부정	0.9970	영화보면서 내가 이거 왜보고있나? 라는 생각이 들었다. 평점에 속은영화!
긍정	0.9614	정의를 위해...일단 봐라 ........신선한 충격이다 ...이런 영화 평생 보기 힘들다
긍정	0.9956	예쁘고 아련한 영화. 나에게도 무지개가 뜨는 날이 오려나. . ㅠㅠ
부정	0.9446	김동욱때문에 본다...
부정	0.8889	영화관에 내친구 둘이랑 나랑 진도나가려는 커플2이랑 5명있었다..
긍정	0.8868	진짜 연기력들 죽이네.. 대사도 여러번 보게 될 거 같다..
부정	0.9968	노잼노잼노잼노잼노잼
부정	0.9787	유오성 명품연기를 기대하면서 봤는데정작 유오성은 엑스트라급 출연비중...유오성 이름 팔아먹네.. ㅡㅡ
긍정	0.9141	그 긴 세월 동안 당신을 사랑하고 있었나봐...
긍정	0.9919	정말 짜릿하고 긴장감 최고였던 영화


In [ ]:
# 직접 입력을 받아서 긍정/부정 예측
while True:
    print("input> ", end="")
    document = str(input())
    if len(document) == 0:
        break
    # TODO: inputs 생성
    inputs = tokenizer(
    document,
    truncation=True,
    max_length=256,
    return_tensors="pt",
    ).to(device)
    # TODO: 추론
    logit = model(**inputs).logits[0]
    prob = F.softmax(logit, dim=-1)
    # TODO: 가장 높은 확률을 정답으로 예측
    y = prob.argmax(dim=-1)
    # 출력
    print(f"{idx2label[y.item()]}\t{prob[y].item():.4f}\t{document}")

input> 긍정
긍정	0.7850	긍정
input> 부정
부정	0.9585	부정
input> 영화
부정	0.7214	영화
input> 이 영화
부정	0.6030	이 영화
input> 너무 좋다
긍정	0.9875	너무 좋다
input> 별로다
부정	0.9980	별로다
input> 최고
긍정	0.9730	최고
input> 싫어
부정	0.9932	싫어
input> 커플
부정	0.5129	커플
input> 사랑
긍정	0.9395	사랑
input> 노잼
부정	0.9977	노잼
input> 